In [1]:
import os
import sys
import h5py
import glob
import time
import numpy as np
import pandas as pd
import pickle as pickle
from pytz import timezone
import astropy.io.fits as pf
from astropy.time import Time
import matplotlib.pyplot as plt
from skyfield.api import load, wgs84, EarthSatellite

In [2]:
### FUNCTIONS NEEDED TO TRANSFORM PIXEL COORDS TO LOOK AT FOV

def world2pix(lst, ra, dec, jd=None, nx=4008, ny=2672, margin=50):
    xwcs, ywcs = world2wcs(wcspars, ra, dec, lst, jd) # Compute wcs-only pixel coordinates
    mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin) # Remove coordinates not within the margins
    xwcs, ywcs = xwcs[mask], ywcs[mask]        
    xpix, ypix = wcs2pix(polpars, xwcs, ywcs) # Convert to actual pixel coordinates.
    return xpix, ypix, mask


def clip_rectangle(x, y, nx, ny, margin):
    maskx = (x > margin) & (x < (nx - margin))
    masky = (y > margin) & (y < (ny - margin))
    return maskx & masky


def world2wcs(wcspars, ra, dec, lst=None, jd=None):
    """ Convert world coordinates to WCS-only coordinates. """    
    if jd is not None:
        ra, dec = j2000_to_equinox(ra, dec, jd)
    w = create_wcs(wcspars, lst)
    xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
    return xwcs, ywcs


def create_wcs(wcspars, lst=None):
    """ Create and astropy WCS instance from a dictionary of parameters. """  
    from astropy import wcs
    
    if lst is not None:
        ra0, dec0 = wcspars['crval']
        ha0 = ra2ha(ra0, wcspars['lst'])
        ra0 = ha2ra(ha0, lst)
        crval = np.array([ra0, dec0])   
    else:
        crval = wcspars['crval']
        
    w = wcs.WCS(naxis=2)
    w.wcs.crpix = wcspars['crpix']
    w.wcs.cdelt = wcspars['cdelt']
    w.wcs.crval = crval
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w.wcs.pc = wcspars['pc']
    return w


def ra2ha(ra, lst):
    """ Convert Right Ascension to Hour Angle. """
    ha = np.mod(lst*15. - ra, 360.)
    return ha

def ha2ra(ha, lst):
    """ Convert Hour Angle to Right Ascension. """    
    ra = np.mod(lst*15. - ha, 360.)
    return ra


def j2000_to_equinox(ra, dec, jd):
    from astropy.time import Time
    from astropy.coordinates import SkyCoord, FK5
    t = Time(jd, scale='utc', format='jd') 
    t.format = 'jyear_str'
    gc = SkyCoord(ra, dec, frame='fk5', unit='deg', equinox='J2000')    
    gc = gc.transform_to(FK5(equinox=t))    
    return gc.ra.value, gc.dec.value


def wcs2pix(polpars, xwcs, ywcs):
    """ Convert WCS-only coordinates to pixel coordinates. """     
    dx, dy = leg2d_eval(xwcs, ywcs, polpars['x_wcs2pix'], polpars['y_wcs2pix'], 
                        polpars['order'], polpars['nx'], polpars['ny'])    
    xpix, ypix = xwcs + dx, ywcs + dy
    return xpix, ypix


def leg2d_eval(x, y, a, b, order=6, nx=4008., ny=2672.):
    """ Evaluate 2-D polynomials. """
    mat, idx1, idx2 = leg2d_mat(x, y, order, nx, ny)
    xres = np.dot(mat, a[idx1, idx2])
    yres = np.dot(mat, b[idx1, idx2])
    return xres, yres  


def leg2d_mat(x, y, order=6, nx=4008., ny=2672.):
    """ Create a matrix for fitting 2-D polynomials of the given order. """ 
    from numpy.polynomial import legendre
    x = np.array(x)
    y = np.array(y)
    mat1 = legendre.legvander(2*(x/nx) - 1, order+1)
    mat2 = legendre.legvander(2*(y/ny) - 1, order+1)
        
    idx1, idx2 = np.indices((order+1, order+1))
    mask = ((idx1 + idx2) <= order)
    idx1, idx2 = idx1[mask], idx2[mask]        
        
    mat = mat1[:,idx1]*mat2[:,idx2]
    return mat, idx1, idx2

In [3]:
def reduce_tles(camid):

    # Load TLEs for all passages
    satfiles = '../test_data/3leComplete.txt'
    with open(satfiles) as f:
        all_tles = f.readlines()
        f.close()   

    # Split TLE list into individual lists for each TLE
    all_tles = [i.strip() for i in all_tles]
    tles = [all_tles[x:x+3] for x in range(0, len(all_tles), 3)]

    # Reduce TLEs to Starlink only
    starlink_tles = []
    for tle in tles:
        if "STARLINK" in tle[0]:
            starlink_tles.append(tle)

    # Obtain satellite passages
    passed_sats = pd.read_pickle(f'../test_data/passages/passed_satellites_20221023{camid}.p')

    # Find any Starlink TLEs in the passages
    idx = []
    flatlist = np.asarray(starlink_tles).flatten()
    for key in passed_sats.keys():
        line1 = passed_sats[key]['TLE line1'].strip()
        i = np.where(flatlist == line1)[0] 
        if i.size > 0:
            idx.append(i[0] - 1) #appending the name of the starlink sat

    # Now have indices for the flattened Starlink TLE list --> divide by 3 to get indices for the original list
    orig_idx = [int(x/3) for x in idx]
    passed_tles = [starlink_tles[i] for i in orig_idx]

    # Remove 0 labeling of first line of TLE because that's the proper format
    for tle in passed_tles:
        tle[0] = tle[0][2:]
    
    return passed_tles

In [4]:
# starlinks = reduce_tles('LSC')

In [5]:
def passage_check(midJD, tles, passages):
        
    idx_reduced = []
    sats = passages[midJD].keys()
    
    satnums = []
    for tle in tles:
        satnums.append(tle[1].split()[1])

    # Cross-reference
    for i, sat in enumerate(sats):
        if sat in satnums:
            idx = satnums.index(sat)
            idx_reduced.append(idx)
  
    return idx_reduced

In [6]:
def get_siteinfo():
    confdir = '../fotos-python3/bringfiles/siteinfo.dat'
    dtype = [('sitename', '|U20'), ('lat', 'float32'), ('lon', 'float32'), ('height', 'float32'), ('ID', '|U2')]
    siteinfo = np.genfromtxt(confdir, dtype=dtype)   
    mask = siteinfo['ID'] == 'LS'
    siteinfo = siteinfo[mask]
    return siteinfo

In [7]:
def check_illumination(midJD, timerange, sat):

    # Check when satellite is illuminated 
    sunlit = sat.at(timerange).is_sunlit(eph)

    # Obtain the indices of the first and last True element for each sequence of True elements 
    sunlit_idx = []
    start_idx = None
    for i, elem in enumerate(sunlit):
        if elem:
            # Checking if the start of a sequence of True elements
            if start_idx is None:
                start_idx = i
        else:
            # Checking if a sequence is just ended
            if start_idx is not None:
                sunlit_idx.append((start_idx, i-1)) # -1 because we're at the first False element after True sequence
                start_idx = None

    # If the last element is True, we need to append its index as well
    if start_idx is not None:
        sunlit_idx.append((start_idx, len(sunlit)-1))
    
    # Getting the times from the indices
    sunlit_times = []
    for idx in sunlit_idx:
        sunlit_times.append([timerange[idx[0]].to_astropy().value, timerange[idx[1]].to_astropy().value])

    # Check if midJD is within each period and flag it if so
    illuminated = False
    for sunlitrng in sunlit_times:
        if midJD >= (sunlitrng[0] - 1/86400) and midJD <= (sunlitrng[1] + 1/86400): #adding a bumper of one second
            illuminated = True
            if illuminated:
                break

    return illuminated

In [16]:
def image_headers(camid):

    images = np.sort(glob.glob(f'../test_data/diff_images/{camid}/diff_*.fits.gz'))
    imgdata = {}
    for img in images:
        header = pf.getheader(img)
        lstseq = img[-19:-11]
        imgdata[lstseq] = {}
        imgdata[lstseq]['JD0'] = header['JD0']
        imgdata[lstseq]['JD1'] = header['JD1']
        imgdata[lstseq]['midLST'] = header['MIDLST']
        imgdata[lstseq]['midJD'] = header['MIDJD']
        imgdata[lstseq]['nx'] = header['XSIZE']
        imgdata[lstseq]['ny'] = header['YSIZE']

    return imgdata

In [17]:
headers = image_headers('LSC')

In [10]:
eph = load('de421.bsp')

def check_each_camid():
    fasts = []
    psgs = []
    tles = []
    camids = ['LSC','LSE']
    pooldicts = [{}] * len(camids)
    
    print('Collecting image information\n')
    for camid in camids:
        fasts.append(h5py.File(f"../fast_20221023{camid}.hdf5", "r"))
        psgs.append(pd.read_pickle(f"../test_data/passages/passages_20221023{camid}.p"))
        tles.append(reduce_tles(camid))
    
    # Define observer
    site = get_siteinfo()
    mascara = wgs84.latlon(latitude_degrees=site[0][1], longitude_degrees=site[0][2], elevation_m=site[0][3])
        
    for camid, fast, starlinks, passages, pool, in zip(camids, fasts, tles, psgs, pooldicts):
        
        images  = glob.glob(f'../test_data/diff_images/{camid}/diff_*.fits.gz')
        lstseqs = [img[-19:-11] for img in images]
            
        for lstseq in lstseqs:
            header = pf.getheader(f'../test_data/diff_images/{camid}/diff_{lstseq}{camid}.fits.gz') 
            midJD  = header['MIDJD']
                
            # Set time of image
            ts = load.timescale()
            t = Time(midJD, format='jd')

            # Obtain passages
            idx_reduced = passage_check(midJD, starlinks, passages)

            if idx_reduced is None:
                print(f'{lstseq}: No starlinks passing MASCARA')
                continue

            print(f'There are {len(idx_reduced)} Starlinks passing overhead {camid} for LSTSEQ={lstseq}')
            # Sun must be low enough below the horizon, otherwise data is not good enough
            observer = mascara + eph['earth']
            sun_pos = observer.at(ts.from_astropy(t)).observe(eph['sun'])
            sun_alt, sun_az, sun_dist = sun_pos.apparent().altaz()

            if sun_alt.degrees <= -18.:
                # Attaining the astrometric solution (depends on FAST file and LSTseq)
                astro = np.where((fast['astrometry/lstseq'][:] // 50) == (int(lstseq) // 50))[0][0]
                order = fast['astrometry/x_wcs2pix'][astro].shape[0]-1
                lst=fast['station/lst'][np.where(fast['station/lstseq'][:]==(fast['astrometry/lstseq'][astro]))[0][0]]
                nx = header['XSIZE']    
                ny = header['YSIZE']

                wcspars = { 'crval' : fast['astrometry/crval'][astro].copy(),
                            'crpix' : fast['astrometry/crpix'][astro].copy(),
                            'cdelt' : [0.02148591731740587,0.02148591731740587],
                            'pc'    : fast['astrometry/pc'][astro].copy(),
                            'lst'   : lst }

                polpars = { 'x_wcs2pix' : fast['astrometry/x_wcs2pix'][astro].copy(),
                            'y_wcs2pix' : fast['astrometry/y_wcs2pix'][astro].copy(),
                            'x_pix2wcs' : fast['astrometry/x_pix2wcs'][astro].copy(),
                            'y_pix2wcs' : fast['astrometry/y_pix2wcs'][astro].copy(),
                            'nx'    : nx,
                            'ny'    : ny,
                            'order' : order }

                jd0 = header['JD0']
                jd1 = header['JD1']
                midLST = header['midLST']
                
                # Timerange
                # jds = np.sort([JD0, midJD, JD1])
                # old = Time(min(jds), format='jd')
                # new = Time(max(jds), format='jd')
                # timerange = np.linspace(old, new, 2, endpoint=True)
                # tr = ts.from_astropy(times)

                for idx in idx_reduced:
                    line1 = starlinks[idx][0]
                    line2 = starlinks[idx][1]
                    line3 = starlinks[idx][2] 
                    sat = EarthSatellite(line2, line3, line1, ts)

                    diff = sat - mascara
                    topocentric = diff.at(ts.from_astropy(t))
                    alt, az, dist = topocentric.altaz()

                    # Criteria check: if satellite is >20 degrees above the horizon
                    if alt.degrees >= 20:

                        # Criteria check: if satellite is illuminated
                        # if sat.at(tr[0]).is_sunlit(eph) and sat.at(tr[1]).is_sunlit(eph):
                        
                        if sat.at(ts.from_astropy(t)).is_sunlit(eph): #maybe wrong since midJD > JD0 and JD1 (why??)
                            ra, dec, radec_dist = topocentric.radec() 
                            radeg = ra._degrees
                            dedeg = dec._degrees
                            ra, dec = j2000_to_equinox(radeg, dedeg, midJD)
                            w = create_wcs(wcspars, midLST)
                            xwcs, ywcs = w.wcs_world2pix(ra, dec, 0)
                            mask = np.isfinite(xwcs) & clip_rectangle(xwcs, ywcs, nx, ny, margin=50)

                            if mask:

                                if lstseq not in pool:
                                    pool[lstseq] = {}

                                if jd0 not in pool[lstseq]:
                                    pool[lstseq][jd0] = {}
                                    pool[lstseq][jd0] = [line2[2:8]]
                                else:
                                    if line2[2:8] not in pool[lstseq][jd0]:
                                        pool[lstseq][jd0].append(line2[2:8])

                                if jd1 not in pool[lstseq]:
                                    pool[lstseq][jd1] = {}
                                    pool[lstseq][jd1] = [line2[2:8]]
                                else:    
                                    if line2[2:8] not in pool[lstseq][jd1]:
                                        pool[lstseq][jd1].append(line2[2:8])

                                if midJD not in pool[lstseq]:
                                    pool[lstseq][midJD] = {}
                                    pool[lstseq][midJD] = [line2[2:8]]
                                else:    
                                    if line2[2:8] not in pool[lstseq][midJD]:
                                        pool[lstseq][midJD].append(line2[2:8])

                                


    for fast in fasts:
        fast.close()
        
    for (pool, camid) in zip(pooldicts, camids):
    
        if len(pool) > 0:
            pickle.dump(pool, open(f'../jd_test/pool_{camid}.p', 'wb'))  



In [11]:
check_each_camid()


There are 2 Starlinks passing overhead LSC for LSTSEQ=48506907
There are 3 Starlinks passing overhead LSC for LSTSEQ=48510542
There are 3 Starlinks passing overhead LSC for LSTSEQ=48509980
There are 1 Starlinks passing overhead LSC for LSTSEQ=48509945
There are 2 Starlinks passing overhead LSC for LSTSEQ=48510920
There are 2 Starlinks passing overhead LSC for LSTSEQ=48506263
There are 2 Starlinks passing overhead LSC for LSTSEQ=48506377
There are 2 Starlinks passing overhead LSC for LSTSEQ=48510733
There are 1 Starlinks passing overhead LSC for LSTSEQ=48510957
There are 2 Starlinks passing overhead LSC for LSTSEQ=48507687
There are 1 Starlinks passing overhead LSC for LSTSEQ=48509045
There are 1 Starlinks passing overhead LSC for LSTSEQ=48508580
There are 6 Starlinks passing overhead LSE for LSTSEQ=48506377
There are 5 Starlinks passing overhead LSE for LSTSEQ=48507687


In [12]:
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")
pool = pd.read_pickle('../jd_test/pool_LSC.p')

In [13]:
pool

{'48510542': {2459876.832331556: ['46760U', '51772U'],
  2459876.832405434: ['46760U', '51772U'],
  2459876.831001776: ['46760U', '51772U']},
 '48506263': {2459876.516233906: ['47763U', '51802U'],
  2459876.516307783: ['47763U', '51802U'],
  2459876.517563686: ['47763U', '51802U']},
 '48506377': {2459876.524655353: ['48366U', '51788U'],
  2459876.524729224: ['48366U', '51788U'],
  2459876.52443372: ['48366U', '51788U']},
 '48510733': {2459876.846441036: ['46770U', '51770U'],
  2459876.846514901: ['46770U', '51770U'],
  2459876.845776163: ['46770U', '51770U']}}

In [10]:
lstseq = '48506377'
header = pf.getheader(f'../test_data/diff_images/LSC/diff_{lstseq}LSC.fits.gz')
JD0 = header['JD0']
JD1 = header['JD1']
midJD = header['midJD']

In [49]:
ts = load.timescale()
jds = np.sort([JD0, midJD, JD1])
old = Time(min(jds), format='jd')
new = Time(max(jds), format='jd')
times = np.linspace(old, new, 2, endpoint=True)
timerange = ts.from_astropy(times)

In [50]:
for time in times:
    t = Time(time, format='jd')
    print(t.iso)

2022-10-24 00:35:11.073
2022-10-24 00:35:36.605


In [ ]:
tles = reduce_tles('LSC')
sat = EarthSatellite(tles[138][1], tles[138][2], tles[138][0], ts)

In [ ]:
#### BETTER WAY
# data_dict = {}
# for camera_id in camids:
#     images = np.sort(glob.glob(f'{subtracted}{date}{camera_id}/*.fits.gz'))
#     lstseqs = np.array([np.int64(img[-19:-11]) for img in images])
#     headers = [pf.getheader(img) for img in images]
#     data_dict[camera_id] = {'LSTSEQs': lstseqs, 'headers': headers}

In [94]:
passages = pd.read_pickle("../test_data/passages/passages_20221023LSC.p")
pool = pd.read_pickle('../selection_pool/pool_LSC.p')

In [95]:
pool['48506263']

{2459876.516233906: ['47763U',
  '51802U',
  '47781U',
  '49442U',
  '48369U',
  '51797U',
  '48026U',
  '49447U',
  '51967U'],
 2459876.516307783: ['47763U',
  '51802U',
  '48369U',
  '51797U',
  '48026U',
  '49447U',
  '51967U'],
 2459876.517563686: ['47763U', '51802U'],
 2459876.516307795: ['47781U',
  '49442U',
  '47773U',
  '48386U',
  '49447U',
  '51797U'],
 2459876.517563697: ['47781U', '49442U'],
 2459876.516233917: ['47773U', '48386U', '49447U', '51797U'],
 2459876.517563709: ['47773U', '48386U', '49447U', '51797U'],
 2459876.517563663: ['48369U', '51797U'],
 2459876.517563703: ['47763U', '48026U', '49447U', '51967U']}